## Assignment 2

In the second assignment, we are going to use a large language model in a retrieval-augmented setup. As an application, we are going to consider a question answering task.

You can use any LLMs you want in this assignment, but your solution must consider at least one open model (e.g. Mistral or one of the Llama models). Optionally, you may compare to a commercial model.

The dataset we will use in this assignment is a simplified version of Natural Questions, which was compiled by Google and consists of real search engine queries about factual questions.

In [2]:
import pandas as pd

nq_data = pd.read_csv('nq_simplified.val.tsv', sep='\t', header=None, names=['question', 'answer', 'gold_context'], quoting=3)
nq_data

,question,answer,gold_context
0,what purpose did seasonal monsoon winds have o...,enabled European empire expansion into the Ame...,The westerlies (blue arrows) and trade winds (...
1,who got the first nobel prize in physics,"Wilhelm Conrad Röntgen, of Germany",The award is presented in Stockholm at an annu...
2,when is the next deadpool movie being released,"May 18, 2018","Though the original creative team of Reynolds,..."
3,where did the idea of fortnite come from,as a cross between Minecraft and Left 4 Dead,"Fortnite is set in contemporary Earth, where t..."
4,which mode is used for short wave broadcast se...,MFSK Olivia,"All one needs is a pair of transceivers, each ..."
...,...,...,...
4284,who challenged the aristotelian model of a geo...,Copernicus,Planets Variations in speed through the zodiac...
4285,when was the miraculous journey of edward tula...,"March 30, 2006",The Miraculous Journey of Edward Tulane-wikipe...
4286,character in macbeth who is murdered and appea...,Lord Banquo,Banquo Thane of Lochaber Macbeth character Thé...
4287,when was as you like it first performed,"uncertain, though a performance at Wilton Hous...",As You Like It-wikipedia As You Like It Jump t...


### Step 1: Evaluating an LLM on Natural Questions

Load an LLM and explore different prompting strategies to try to make it answer the questions in the dataset. As a benchmark, you can use the ROUGE-1 precision/recall/F1 scores.

In [3]:
def rouge1(gold, predicted):
  assert(len(gold) == len(predicted))
  n_p = 0
  n_g = 0
  n_c = 0
  for g, p in zip(gold, predicted):
    g = set(cleanup(g).strip().split())
    p = set(cleanup(p).strip().split())
    n_g += len(g)
    n_p += len(p)
    n_c += len(p.intersection(g))
  pr = n_c / n_p
  re = n_c / n_g
  if pr > 0 and re > 0:
    f1 = 2*pr*re/(pr + re)
  else:
    f1 = 0.0
  return pr, re, f1

def cleanup(text):
  text = text.replace(',', ' ')
  text = text.replace('.', ' ')
  return text

In [4]:
from transformers import pipeline

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]
chatbot = pipeline("text-generation", model="mistralai/Mistral-7B-Instruct-v0.3")
chatbot(messages)

/home/andre/Repos/wasp/WASP DL4NLP 2024/Assignment 2/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 